![imagenes](logo.png)

In [2]:
import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import *
import os

In [3]:
## Nos dirigimos a nuestra carpeta de trabajo
os.chdir("C:\\Users\\hp master\\OneDrive\\Escritorio\\delitos")

In [4]:
## Observamos qué tenemos en esa carpeta
os.listdir()

['delitos_2012.csv',
 'delitos_2013.csv',
 'delitos_2014.csv',
 'delitos_2015.csv',
 'delitos_2016.csv',
 'delitos_2017.csv',
 'delitos_2018.csv',
 'delitos_2019.csv',
 'delitos_2020.csv',
 'delitos_2021.csv',
 'descriptor_delitos.csv',
 'descriptor_entidades.csv']

In [5]:
## Leemos las tablas de delitos
delitos = [pd.read_csv(x) for x in os.listdir() if x.startswith("delitos")]

In [6]:
## Mostramos la primer tabla
delitos[0]

,entidad,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,4325,63117,65608,26673,32467,2397,12422,19107,52892,33144,13238,155.0,7533,374.0,NaN
1,2,53101,125100,146944,156090,135178,1941,37776,46993,214567,82699,33323,1128.0,5990,NaN,341.0
2,3,6109,24582,19784,27540,8094,3023,8046,21432,23850,23136,5669,331.0,1568,156.0,132.0
3,4,642,16428,20120,20517,23402,6763,6619,15702,25519,41469,12456,NaN,4714,1210.0,910.0
4,5,13662,70668,73930,53498,67094,3596,20410,22978,35124,35549,16586,1772.0,8311,NaN,NaN
5,6,5047,15337,12729,15046,8411,2544,7721,6493,47003,9833,3723,192.0,174,355.0,502.0
6,7,1717,29559,48524,74234,93405,18171,7800,32578,69132,44713,24271,98.0,4582,450.0,NaN
7,8,46786,142051,66602,87663,120161,14607,68784,49308,216619,86056,37165,5602.0,13446,4694.0,4153.0
8,9,59305,305377,403522,104286,1183773,15590,275095,126797,714753,248338,110061,16215.0,109376,9206.0,528.0
9,10,7761,35586,34404,25044,42167,2312,8468,14999,111644,42871,12206,1150.0,4760,1424.0,122.0


In [7]:
## Leemos la tabla descriptor_delitos y la mostramos
descriptor_delitos = pd.read_csv("descriptor_delitos.csv",encoding="latin-1")
descriptor_delitos

,cod_del,Delito
0,1,Robo total de vehículo
1,2,Robo parcial de vehículo
2,3,Vandalismo
3,4,Robo en casa
4,5,Robo en la calle
5,6,Otro tipo de robo
6,7,Fraude bancario
7,8,Fraude al consumidor
8,9,Extorsión
9,10,Amenazas verbales


In [8]:
## Leemos la tabla descriptor_entidades y la mostramos
descriptor_entidades = pd.read_csv("descriptor_entidades.csv",encoding="latin-1")

In [9]:
(delitos[0] >> 
    mutate(año = 0+2012) >>
    gather("cod_del",
           "total",
           _[1:16]) >>
    left_join(_,descriptor_entidades, on = "entidad") >>
    mutate(cod_del = _.cod_del.astype("int"), total = _.total.astype("Int64")) >>
    left_join(_,descriptor_delitos, on = "cod_del") >>
    select(_.año, _.entidad, _.Entidad, _.Delito, _.total)
    )

,año,entidad,Entidad,Delito,total
0,2012,1,Aguascalientes,Robo total de vehículo,4325
1,2012,2,Baja California,Robo total de vehículo,53101
2,2012,3,Baja California Sur,Robo total de vehículo,6109
3,2012,4,Campeche,Robo total de vehículo,642
4,2012,5,Coahula,Robo total de vehículo,13662
...,...,...,...,...,...
475,2012,28,Tamaulipas,Otros,2225
476,2012,29,Tlaxcala,Otros,1203
477,2012,30,Veracruz,Otros,504
478,2012,31,Yucatán,Otros,5149


In [13]:
def ordenado(x):
    auxiliar = (delitos[x] >> 
    mutate(año = x+2012) >>
    gather("cod_del",
           "total",
           _[1:16]) >>
    left_join(_,descriptor_entidades, on = "entidad") >>
    mutate(cod_del = _.cod_del.astype("int"), total = _.total.astype("Int64")) >>
    left_join(_,descriptor_delitos, on = "cod_del") >>
    select(_.año, _.entidad, _.Entidad, _.Delito, _.total)
    )
    return auxiliar

In [19]:
pd.concat([ordenado(x) for x in range(0,10)]) >> top_n(10,wt=_.total) >> arrange(-_.total)

,año,entidad,Entidad,Delito,total
142,2013,15,Estado de México,Robo en la calle,4917122
142,2014,15,Estado de México,Robo en la calle,4379121
142,2018,15,Estado de México,Robo en la calle,3524963
142,2017,15,Estado de México,Robo en la calle,3411601
142,2015,15,Estado de México,Robo en la calle,3277422
142,2012,15,Estado de México,Robo en la calle,3236823
142,2016,15,Estado de México,Robo en la calle,3142406
142,2019,15,Estado de México,Robo en la calle,3093425
270,2013,15,Estado de México,Extorsión,2436631
142,2021,15,Estado de México,Robo en la calle,2323408
